# Infer model on array

---

## Imports

In [1]:
import torch
import xbatcher
import xarray as xr
import numpy as np

## Toy data

In [2]:
data = xr.DataArray(
    data=np.random.rand(100, 100, 10),
    dims=("x", "y", "t")
)
data

<xarray.DataArray (x: 100, y: 100, t: 10)> Size: 800kB
array([[[0.44893786, 0.42089057, 0.62811935, ..., 0.77825309,
         0.71424518, 0.27544911],
        [0.18512313, 0.22459034, 0.32941286, ..., 0.72642438,
         0.453367  , 0.79092317],
        [0.75001122, 0.75201415, 0.49362845, ..., 0.12351849,
         0.07800031, 0.5129466 ],
        ...,
        [0.79767373, 0.04314154, 0.42138962, ..., 0.64572546,
         0.68486366, 0.26731422],
        [0.55990821, 0.66252347, 0.35829701, ..., 0.2245369 ,
         0.210521  , 0.3942034 ],
        [0.26561369, 0.32802417, 0.33247419, ..., 0.89834246,
         0.85079973, 0.30808448]],

       [[0.49745507, 0.47563665, 0.25939342, ..., 0.95437792,
         0.3529035 , 0.22432583],
        [0.93810888, 0.36236317, 0.90214027, ..., 0.52733391,
         0.08590167, 0.39300478],
        [0.39701346, 0.87476725, 0.07576221, ..., 0.6206168 ,
         0.590455  , 0.18998422],
...
        [0.59384608, 0.60548133, 0.86872946, ..., 0.18636072,
         0.23673955, 0.47465824],
        [0.53055158, 0.63400677, 0.74856164, ..., 0.57724106,
         0.12961194, 0.23441086],
        [0.04898592, 0.96045848, 0.96103484, ..., 0.11542319,
         0.03916002, 0.73413531]],

       [[0.71738595, 0.97645637, 0.98795853, ..., 0.27255291,
         0.01424937, 0.08837012],
        [0.64385468, 0.25559843, 0.55311384, ..., 0.45737679,
         0.6211287 , 0.40359951],
        [0.42621607, 0.309378  , 0.1063637 , ..., 0.23134538,
         0.04260683, 0.64022588],
        ...,
        [0.8286403 , 0.18384525, 0.80661243, ..., 0.78306387,
         0.29219244, 0.22387774],
        [0.65402132, 0.56136263, 0.86077124, ..., 0.3128178 ,
         0.64353248, 0.52959765],
        [0.76868123, 0.67687677, 0.42182969, ..., 0.86638394,
         0.68939672, 0.14375773]]], shape=(100, 100, 10))
Dimensions without coordinates: x, y, t

## Simple model

In [3]:
class MeanAlongDim(torch.nn.Module):
    def __init__(self, ax):
        super(MeanAlongDim, self).__init__()
        self.ax = ax

    def forward(self, x):
        return torch.mean(x, self.ax)

## Batch generator, dataset

In [4]:
from xbatcher.loaders.torch import MapDataset

bgen = xbatcher.BatchGenerator(
    data,
    input_dims=dict(x=10, y=10),
    input_overlap=dict(x=5, y=5),
)

ds = MapDataset(bgen)

inp = next(iter(ds))

# Check the input/output size of the first example
print("Input shape:", inp.shape)

mad = MeanAlongDim(-1)
print("Output shape:", mad(inp).shape)

Input shape: torch.Size([10, 10, 10])
Output shape: torch.Size([10, 10])


In [5]:
assert torch.allclose(mad(inp), torch.mean(inp, -1))

## Inference function

In [6]:
%run ./functions.ipynb

In [7]:
out = predict_on_array(
    dataset=ds, 
    model=mad, 
    output_tensor_dim=dict(y=20, x=10, x_new=5),
    new_dim=["x_new"],
    resample_dim=["x", "y"]
)
print(out.shape)

(200, 100, 5)
